In [ ]:
import config
import tweepy
import pandas as pd
import requests
import datetime

In [ ]:
today = datetime.datetime.today()
tomorrow = today + datetime.timedelta(days=1)
dayAfter = today + datetime.timedelta(days=2)
dates = [today.strftime("%d-%m-%Y"),tomorrow.strftime("%d-%m-%Y"),dayAfter.strftime("%d-%m-%Y")]
failedTweet = [1389616776728645636]

In [ ]:
clean = False
df = pd.read_csv("UserData.csv")

In [ ]:
auth = tweepy.OAuthHandler(config.key,config.scret_key)
auth.set_access_token(config.access_token,config.access_token_secret)
api = tweepy.API(auth)

In [ ]:
df = pd.read_csv("UserData.csv")

sinceID = max(df.TweetID.max(),max(failedTweet))
for mention in tweepy.Cursor(api.mentions_timeline,since_id=sinceID).items():
    splitt = mention.text.split(" ")
    username = mention.user.screen_name
    tweetID = mention.id

    if clean:
        try:
            api.destroy_status(tweetID)
            print("Clean: Deleted")
        except:
            print("Clean: Skipped")
            continue
    else:
        try:
            pin = splitt[2]
            age = int(splitt[3].split("+")[0])
            reply = "@" + username + " You have been successfuly subscribed. I will be reminding you when vaccine slot is available at your pincode " + pin
            api.update_status(reply,tweetID)
            df = df.append({ "Username": username,"TweetID": tweetID, "Pin": pin, "Age": age},ignore_index=True)
            print(pin,age,username,tweetID)
        except:
            reply = "@" + username + " " + 'There seems to be an error. Please make sure your tweet is in the following format ' + "errorid:" + datetime.datetime.now().strftime("%Y%m%d%H%M%S")
            file = open("ss.jpeg" , "rb")
            r1 = api.media_upload(filename="ss.jpeg",file=file)
            api.update_status(reply,tweetID,media_ids=[r1.media_id_string])
            print("Wrong tweet")
            failedTweet.append(tweetID)
            continue

df.to_csv("UserData.csv",index=False)

In [ ]:
baseURL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/findByPin?"
df = pd.read_csv("UserData.csv")
for date in dates:
    for pin in df["Pin"].unique():

        completeURL = baseURL+"date="+date + "&pincode=" + str(pin)
        try:
            sessions = requests.get(completeURL).json()["sessions"]
        except:
            print("Failed to get new sessions")

        if sessions:
            vaccines = {"COVISHIELD" : False, "COVAXIN" : False}
            minAge = {18:False,45:False}
            availableCapacity = sys.maxsize

            for session in sessions:
                availableCapacity = min(availableCapacity,int(session["available_capacity"]))
                vaccines[session["vaccine"]] = True
                minAge[session["min_age_limit"]] = True
            if minAge[18]:
                tweetAt(availableCapacity,vaccines,pin,date,18)
            if minAge[45]:
                tweetAt(availableCapacity,vaccines,pin,date,45)

        else:
            continue

In [118]:
def tweetAt(availableCapacity,vaccines,pin,date,age):

    reply = "Atleast " + str(availableCapacity) + " slot(s) for " + " and ".join([v for v,b in vaccines.items() if b == True]) + " is available for age " + str(age) + "+ at pin " + str(pin) + " for " + date
    newDF = df.dropna()
    tweetIDs = newDF.loc[(df["Age"]==age) & (df["Pin"]==pin),["TweetID"]]

    for tweetID in tweetIDs.TweetID:
        username = df.loc[df["TweetID"]==tweetID,["Username"]].Username.item()
        tweet = "@" + username + " " + reply
        print(tweet)
        api.update_status(tweet,tweetID)

In [119]:
api.update_status("I AM ALIVE")

Status(_api=<tweepy.api.API object at 0x1166225e0>, _json={'created_at': 'Tue May 04 16:32:06 +0000 2021', 'id': 1389618850476396545, 'id_str': '1389618850476396545', 'text': 'I AM ALIVE', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://help.twitter.com/en/using-twitter/how-to-tweet#source-labels" rel="nofollow">getThisSlot</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1389340423332659206, 'id_str': '1389340423332659206', 'name': 'CowinHelper', 'screen_name': 'getmyslot', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 0, 'listed_count': 0, 'created_at': 'Mon May 03 22:05:54 +0000 2021', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'veri